In [1]:
import pandas as pd
import numpy as np

In [2]:
age_at_scan = pd.ExcelFile("/home/a.kidane@mydre.org/brickstudy/secret_data/BRICK_datums_scans_clean_17072024 _versie_2.xlsx", engine = "openpyxl")


In [3]:
sheet_names = age_at_scan.sheet_names

In [4]:
print("Sheet names:", sheet_names)

Sheet names: ['Planbestand', 'Intekenlijst BRICK', 'MRI scan planning overzicht', 'NPO planning overzicht', 'Dag van MRI+NPO overzicht ']


In [5]:
age_at_scan_2 = pd.read_excel("/home/a.kidane@mydre.org/brickstudy/secret_data/BRICK_datums_scans_clean_17072024 _versie_2.xlsx", sheet_name = "Dag van MRI+NPO overzicht ", engine = "openpyxl")
brick_gender = pd.read_excel("/home/a.kidane@mydre.org/brickstudy/secret_data/Gender_BRICKers23072024.xlsx", sheet_name = "Sheet1", engine = "openpyxl")
print(age_at_scan_2.head())

  BRICK-nummer Genotype  Leeftijd_bij_scan Datum_MRI_T0         Datum_NPO_T0
0   BRICK_001      HbSS                 16   2022-06-20  2022-07-28 00:00:00
1   BRICK_002      HbSC                 15   2022-06-20  2022-07-29 00:00:00
2    BRICK_003     HbSS                 13   2022-06-20  2022-07-08 00:00:00
3    BRICK_004     HbSS                  9   2022-07-04  2022-08-10 00:00:00
4    BRICK_005     HbSC                  9   2022-12-14  2023-02-09 00:00:00


In [6]:
#just to be sure, trim whitespace in the column names and the column "BRICK-nummer"
age_at_scan_2.columns = age_at_scan_2.columns.str.strip()

age_at_scan_2["BRICK-nummer"] = age_at_scan_2["BRICK-nummer"].str.strip()
print(age_at_scan_2.head())

  BRICK-nummer Genotype  Leeftijd_bij_scan Datum_MRI_T0         Datum_NPO_T0
0    BRICK_001     HbSS                 16   2022-06-20  2022-07-28 00:00:00
1    BRICK_002     HbSC                 15   2022-06-20  2022-07-29 00:00:00
2    BRICK_003     HbSS                 13   2022-06-20  2022-07-08 00:00:00
3    BRICK_004     HbSS                  9   2022-07-04  2022-08-10 00:00:00
4    BRICK_005     HbSC                  9   2022-12-14  2023-02-09 00:00:00


In [7]:
#remove the "00:00:00" by removing characters in "Datum_NPO_T0"
age_at_scan_2["Datum_NPO_T0"] = age_at_scan_2["Datum_NPO_T0"].astype(str)
age_at_scan_2["Datum_NPO_T0"] = age_at_scan_2["Datum_NPO_T0"].str.strip()
age_at_scan_2["Datum_NPO_T0"] = age_at_scan_2["Datum_NPO_T0"].str[:-9]

print(age_at_scan_2.head())

  BRICK-nummer Genotype  Leeftijd_bij_scan Datum_MRI_T0 Datum_NPO_T0
0    BRICK_001     HbSS                 16   2022-06-20   2022-07-28
1    BRICK_002     HbSC                 15   2022-06-20   2022-07-29
2    BRICK_003     HbSS                 13   2022-06-20   2022-07-08
3    BRICK_004     HbSS                  9   2022-07-04   2022-08-10
4    BRICK_005     HbSC                  9   2022-12-14   2023-02-09


In [8]:
#rename second column to gender
brick_gender.rename(columns={brick_gender.columns[1]: "Gender"}, inplace=True)
print(brick_gender.head())

  Participant ID Gender
0      BRICK-001      V
1      BRICK-002      M
2      BRICK-003      V
3      BRICK-004      M
4      BRICK-005      M


In [9]:
#now make the hyphen a underscore up age at scan csv with this table
brick_gender["Participant ID"] = brick_gender["Participant ID"].str.replace('-',"_")
print(brick_gender.head())

  Participant ID Gender
0      BRICK_001      V
1      BRICK_002      M
2      BRICK_003      V
3      BRICK_004      M
4      BRICK_005      M


In [10]:
#prepare merging, first rename Participant ID columns without spaces
age_at_scan_2.rename(columns={"BRICK-nummer" : "Participant_ID"}, inplace=True)
brick_gender.rename(columns={"Participant ID" : "Participant_ID"}, inplace=True)

In [11]:
BRICK_volumetric_dems = pd.merge(age_at_scan_2, brick_gender, on="Participant_ID", how="left")
print(BRICK_volumetric_dems.head())

  Participant_ID Genotype  Leeftijd_bij_scan Datum_MRI_T0 Datum_NPO_T0 Gender
0      BRICK_001     HbSS                 16   2022-06-20   2022-07-28      V
1      BRICK_002     HbSC                 15   2022-06-20   2022-07-29      M
2      BRICK_003     HbSS                 13   2022-06-20   2022-07-08      V
3      BRICK_004     HbSS                  9   2022-07-04   2022-08-10      M
4      BRICK_005     HbSC                  9   2022-12-14   2023-02-09      M


In [21]:
#now merge this one with the volumetric data csv, drop first index column
brick_volumetric = pd.read_csv("/home/a.kidane@mydre.org/brickstudy/secret_data/brain_volumes_from_freesurfer_no_qc.csv", index_col=0)

In [22]:
#rename first column and change hyphens into underscores
brick_volumetric.rename(columns={"Participant ID" : "Participant_ID"}, inplace=True)
brick_volumetric["Participant_ID"] = brick_gender["Participant_ID"].str.replace('-',"_")

In [23]:
#merge the demographic data and demographic data

brick_volumetric_dems_00 = pd.merge(brick_volumetric,BRICK_volumetric_dems, on="Participant_ID", how="left")

print(brick_volumetric_dems_00.head())

  Participant_ID  Left-Lateral-Ventricle  Left-Inf-Lat-Vent  \
0      BRICK_001                  3003.6              236.3   
1      BRICK_002                  3423.4              203.2   
2      BRICK_003                  4516.6             1085.6   
3      BRICK_004                  6298.0              438.4   
4      BRICK_005                 12589.6              352.9   

   Left-Cerebellum-White-Matter  Left-Cerebellum-Cortex  Left-Thalamus-Proper  \
0                       13804.5                 56622.6                6919.7   
1                       12140.0                 50067.4                6404.7   
2                       12862.1                 51962.8                7410.2   
3                       13439.1                 60803.3                8451.7   
4                       13078.8                 58877.9                7250.1   

   Left-Caudate  Left-Putamen  Left-Pallidum  3rd-Ventricle  ...  \
0        2986.4        4495.1         1846.9          560.1  ...  

In [24]:
file_path = "/home/a.kidane@mydre.org/brickstudy/secret_data/brick_volumetric_dems_01.csv"
brick_volumetric_dems_00.to_csv(file_path, index=False)